# Yeast cells detection evaluation notebook

Evaluates the performance of segmentation and tracking on the YIT ground truth data set. 

Performances are displayed as calibration curves over the `segmentation_threshold`, and as a calibration heatmap over the tracking hyperparameters `dmax` and `epsilon`.

In [ ]:
try:
  from yeastcells import data, features, yit, segmentation, tracking, evaluation
except ImportError:
  # Outside of colab, it is necesary to manually install scikit-learn, seaborn, torch and torchvision, e.g.:
  # !pip3 install scikit-learn torch torchvision seaborn
  !pip3 install -U git+https://github.com/ymzayek/yeastcells-detection-maskrcnn.git
  from yeastcells import data, features, yit, segmentation, tracking, evaluation

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.auto import tqdm
from download import download
from skimage.io import imread
from itertools import product
import matplotlib.pyplot as plt
from multiprocessing import Pool
from scipy.sparse import coo_matrix
from sklearn.exceptions import EfficiencyWarning

%matplotlib inline

In [ ]:
# Tested thresholds
thresholds = np.arange(.8, 1, 0.005)

# Tested hyperparameter values for tracking
dmaxs = np.arange(2, 7, 1)
epsilons = np.arange(0.35, 0.91, 0.05)

# Selected threshold
segmentation_threshold=0.955

testset_names = [
 'TestSet1',
 'TestSet2',
 'TestSet3',
 'TestSet4',
 'TestSet5',
 'TestSet6',
 'TestSet7',
#  'TestSet8',
#  'TestSet9',
#  'TestSet10',
]

## Download model and data

In [ ]:
pipeline_path = f'./yeastcells-detection-maskrcnn'
model_filename = f'{pipeline_path}/model_final.pth'

download(
    'https://datascience.web.rug.nl/models/yeast-cells/mask-rcnn/v1/model_final.pth', model_filename)

download(
    'https://datascience.web.rug.nl/YeaZ_results.zip',
    f'{pipeline_path}/YeaZ_results/YeaZ_results.zip')

download(
    'https://datascience.web.rug.nl/YIT-Benchmark2.zip', 
    f'{pipeline_path}/YIT-Benchmark2/YIT-Benchmark2.zip')

!cd '{pipeline_path}' && unzip -o -qq 'YIT-Benchmark2/YIT-Benchmark2.zip' -d YIT-Benchmark2
!cd '{pipeline_path}' && unzip -o -qq 'YeaZ_results/YeaZ_results.zip' -d YeaZ_results

# We've precomputed the outcomes of the YeaZ model on the YIT data
def load_test_yeaz_detections(path, testset_name = 'TestSet1'):
  # one mask per frame, with integer encoded segmentation.
  masks = data.read_tiff_mask(f'{path}/YeaZ_results/YeaZ_masks_{testset_name}.tiff')

  # a dataframe with every detection per frame seperately
  yeaz_detections = pd.DataFrame([
    {'frame': frame, 'cell': cell}
    for frame, mask in enumerate(masks)
    for cell in np.unique(mask)
    if cell > 0
  ])

  # create one-hot encoded masks, like we get from the Mask-RCNN, such that they
  # fit our evaluation pipeline.
  masks = (
      masks[yeaz_detections['frame']] # repeat frames per detection of a frame
      == yeaz_detections['cell'].values[:, None, None] # one-hot encode
  )
  # we could use the dataframe's index, but a seperate column is safer to mutations
  yeaz_detections['mask'] = np.arange(len(masks))
  return masks, yeaz_detections

## Performance

### YeaZ vs. our Mask R-CNN pipeline

Comparing YeaZ and our model w.r.t. different metrics. We've used different thresholds per test set, as shown in the section 'Segmentation theshold optimization'. We've also determined general values for dmax and epsilon below.

In [ ]:
optimal_thresholds = {
    'TestSet1': 0.955, 'TestSet2': 0.975, 'TestSet3': 0.9, 'TestSet4': 0.9,
    'TestSet5': 0.9, 'TestSet6': 0.9, 'TestSet7': 0.9}

In [ ]:
metrics = []
import time

for testset_name in tqdm(testset_names, desc='Overall evaluation progress'):
  image = yit.get_test_movie(pipeline_path, testset_name)
  
  ground_truth = yit.get_ground_truth(pipeline_path, testset_name)
  
  # YeaZ results
  yeaz_masks, yeaz_detections = load_test_yeaz_detections(pipeline_path, testset_name)
  
  # Mask R-CNN results
  threshold = optimal_thresholds[testset_name]
  maskrcnn_detections, maskrcnn_masks = segmentation.get_segmentation(
      image, model_filename, seg_thresh=threshold, device='cuda:0')
  
  # Tracking Mask R-CNN results via DBSCAN clustering
  maskrcnn_detections = tracking.track_cells(maskrcnn_detections, maskrcnn_masks,
                                             dmax=5, min_samples=3, eps=0.6, device='cuda:0')
  
  metrics.extend([
      {'test set': testset_name, 'task': task, 'model': model,
       'metric': metric, 'value': value}
      for task, get_metrics in {
          'tracking': evaluation.get_tracking_metrics,
          'segmentation': evaluation.get_segmention_metrics}.items()
      for model, detections, masks in [
        ('YeaZ', yeaz_detections, yeaz_masks),
        ('Mask R-CNN', maskrcnn_detections, maskrcnn_masks),
      ]
      for metric, value in evaluation.calculate_metrics(get_metrics(
        ground_truth, detections, masks)).items()
  ])

results = pd.DataFrame(metrics)

In [ ]:
results.pivot(('test set', 'model'), columns=('task', 'metric'), values='value')

### Segmentation theshold optimization

In [ ]:
metrics = []

for testset_name in tqdm(testset_names, desc='Overall evaluation progress'):
  image = yit.get_test_movie(pipeline_path, testset_name)
  ground_truth = yit.get_ground_truth(pipeline_path, testset_name)
  
  # Mask R-CNN results
  maskrcnn_detections, maskrcnn_masks = segmentation.get_segmentation(
      image, model_filename, seg_thresh=float(thresholds.min()), device='cuda:0')
  
  for i, threshold in enumerate(tqdm(thresholds,
                                     desc=f'Progress for {testset_name}',
                                     leave=False)):
    
    # Filter out detections according to threshold.
    above_threshold = maskrcnn_detections['segmentation_score'] >= threshold
    maskrcnn_detections_above_threshold = maskrcnn_detections[above_threshold].copy()
    
    # Tracking Mask R-CNN results via DBSCAN clustering
    maskrcnn_detections_above_threshold = tracking.track_cells(
      maskrcnn_detections_above_threshold, maskrcnn_masks,
      dmax=5, min_samples=3, eps=0.6, device='cuda:0')
    
    metrics.extend([
        {'test set': testset_name, 'task': task, 'metric': metric,
         'value': value, 'threshold': threshold, 'model': 'Mask R-CNN'}
        for task, get_metrics in {
            'tracking': evaluation.get_segmention_metrics,
            'segmentation': evaluation.get_tracking_metrics}.items()
        for metric, value in evaluation.calculate_metrics(get_metrics(
          ground_truth, maskrcnn_detections_above_threshold, maskrcnn_masks
        )).items()
    ])
threshold_results = pd.DataFrame(metrics)

#### Results

The performance of segmentation and tracking shown as calibration curves with respect to the threshold

In [ ]:
for testset_name, testset_results in tqdm(threshold_results.groupby('test set')):
  fig, axes = plt.subplots(1, 2, figsize=(20, 5))
  for (task, task_results), axis in zip(testset_results.groupby('task'), axes.ravel()):
    optimal_threshold = optimal_thresholds[testset_name]
    axis.vlines(optimal_threshold, 0, 1, label='selected threshold')
    
    sns.lineplot(data=testset_results, x='threshold', y='value', hue='metric', ax=axis, ci=None)
    axis.set_title(task); axis.set_ylim(0, 1)
  plt.suptitle(testset_name)

#### Export

Export as spreadsheet.

In [ ]:
threshold_results.pivot(index=['threshold'],
                        columns=['test set', 'task', 'metric'],
                        values='value').to_excel('threshold-tuning-results.xlsx')

try:
  from google.colab.files import download
  download('threshold-tuning-results.xlsx')
except ImportError:
  print('Not on Google Colab, won\'t start download')

### Tuning `dmax` and `epsilon` hyperparameters for tracking

In [ ]:
import importlib
importlib.reload(tracking)

In [ ]:
metrics = []

import time

for testset_name in tqdm(testset_names, desc='Overall evaluation progress'):
  image = yit.get_test_movie(pipeline_path, testset_name)
  ground_truth = yit.get_ground_truth(pipeline_path, testset_name)
  
  # Mask R-CNN results
  threshold = optimal_thresholds[testset_name]
  maskrcnn_detections, maskrcnn_masks = segmentation.get_segmentation(
      image, model_filename, seg_thresh=threshold, device='cuda:0')
  
  for dmax in tqdm(dmaxs, desc=f'Progress for {testset_name}', leave=False):
    # Distances only depend on dmax, and hence can be reused for all epsilons
    t0 = time.time()
    distances = tracking.get_distances(maskrcnn_detections, maskrcnn_masks,
                                       dmax=dmax, device='cuda:0')
    print(1, time.time()-t0)
    t0, t1 = 0, 0
    for epsilon in epsilons:
      # Tracking Mask R-CNN results via DBSCAN clustering
      t0 -= time.time()
      maskrcnn_detections = tracking.track_cells(
        maskrcnn_detections, maskrcnn_masks,
        dmax=dmax, min_samples=3, eps=epsilon, device='cuda:0',
        distances = distances)
      t0 += time.time()
      t1 -= time.time()
      metrics.extend([
        {'test set': testset_name, 'task': task, 'model': 'Mask R-CNN',
         'threshold': threshold, 'dmax': dmax, 'eps': epsilon,
         'metric': metric, 'value': value}
        for task, get_metrics in {
            'tracking': evaluation.get_segmention_metrics,
            'segmentation': evaluation.get_tracking_metrics}.items()
        for metric, value in evaluation.calculate_metrics(get_metrics(
          ground_truth, maskrcnn_detections, maskrcnn_masks
        )).items()
      ])
      t1 += time.time()
    print(2, t0, t1)
    

tracking_results = pd.DataFrame(metrics)

#### Results

Several performance metrics represented by bars or a heatmap for different `eps` and `dmax` values.

In [ ]:
%matplotlib inline


for (testset_name, task), set_task_rows in tqdm(tracking_results.groupby(['test set', 'task'])):
  axes = plt.subplots(2, 4, figsize=(20, 10))[1].T
  for (metric, rows), (ax0, ax1) in zip(set_task_rows.groupby('metric'), axes):
    rows = rows.rename({'value': metric}, axis=1)
    sns.barplot(data=rows.rename({'value': metric}, axis=1),
                hue='dmax', x='eps', y=metric, palette='rainbow', ax=ax0)
    ax0.set_xticklabels([f'{eps:0.2f}' for eps in epsilons], rotation=90)
    ax1.set_ylim(0, 1)

    # Show as pecentages
    rows = (100 * rows.pivot('dmax', columns=['eps'], values=metric)).astype(int)
    sns.heatmap(data=rows, annot=True, ax=ax1, vmin=80, vmax=100, fmt='d')
    ax1.set_xticklabels([f'{eps:0.2f}' for eps in epsilons], rotation=90)
    ax1.set_title(metric)
  plt.tight_layout()
  plt.title(f'{testset_name} - {task}')

#### Export

Save and download these results as a spreadsheets.

In [70]:
tracking_results.pivot(
  index=['test set', 'dmax', 'eps'],
  columns=['task', 'metric'], values='value').to_excel('tracking-tuning-results.xlsx')

try:
  from google.colab.files import download
  download('tracking-tuning-results.xlsx')
except ImportError:
  print('Not on Google Colab, won\'t start download')

Not on Google Colab, won't start download
